In [ ]:
import wget
import requests
import os
import numpy as np
import pandas as pd
import sys
import re
import ssl

In [ ]:
basePath=''
flagPFcode=True
pfam='PF00806'
param=''
filename=''
rangeSeq=10
PDBCodeStructure=[]
maxLongSeq=550
minLongSeq=20

In [ ]:
if(len(sys.argv) > 1):    
    param=sys.argv[1]
    posDot=param.find('.')
    if(param=="default"):
        print("Default config")
    elif (posDot>0):
        extensionFile=param[posDot:]
        if(extensionFile==".txt"):
            print("Input file is a sequence. Exiting this process for family fragments.")
            sys.exit()
            
        flagPFcode=False
        filename=param
    else:
        pfam=param

if(len(sys.argv) > 2): 
    param2=sys.argv[2]
    dirAux=os.path.join(param2, "target")
    isdir = os.path.isdir(dirAux) 
        
    if not isdir:        
        try:
            os.makedirs(dirAux)
        except:
            print("Error with directory: "+dirAux)
            sys.exit()

    basePath=os.path.join(param2, "")


In [ ]:
#quitar los cambios de linea en una secuencia
def preProcessText(data, initPoint, endPoint):
    seq=data
    pos=seq.find('\n')
    
    after=seq[pos+1:]
    after=after.replace('\n','')
    
    if(len(after)-2*rangeSeq>=minLongSeq):
        after=after[initPoint+rangeSeq:endPoint-rangeSeq+1]
    
    print("Sequence length: "+str(len(after)))

    if((len(after)<=maxLongSeq) and (len(after)>=minLongSeq)):
        print("Sequence length ("+str(len(after))+") between range valid to prediction - ["+str(minLongSeq)+", "+str(maxLongSeq)+"]")
        before=seq[:pos+1]
        full=before+after
        return (True, full)
    
    print("Sequence length ("+str(len(after))+") not valid to prediction. Range valid - ["+str(minLongSeq)+", "+str(maxLongSeq)+"]")
    return (False, "")

In [1]:
#Generamos el archivo
def response2fasta (nameFile,response, initPoint, endPoint):
    print("Preprocessing "+ nameFile)
    rpta=preProcessText(response.text, initPoint, endPoint)
    
    if(rpta[0]):
        text_file = open(nameFile, "w")
        text_file.write(rpta[1])
        text_file.close()
        return True
    
    return False

In [ ]:
def correctSeq(x):
    x=x.replace('.','')
    x=x.replace('-','')
    return x.upper()

In [ ]:
#Generamos el archivo
def df2fastaIndividual (dfIn, type):
    if(type==1):
        preName=basePath+'target/is_'
    else:
        preName=basePath+'target/target_'

    #Organizamos la información tal cual la estructura inicial
    dfAux=pd.DataFrame()
    dfAux["agrupado"]=dfIn["encabezado"]+ os.linesep+ dfIn["secuencia"]+ os.linesep
   
    listContent = dfAux['agrupado'].tolist()
    listName = dfIn['id'].tolist()
    
    for i in range(len(listName)):
        nameFile=preName+listName[i]+'_'+str(i)+'.fasta' #is: seq independient
        print("Generating file: "+ nameFile)
        text_file = open(nameFile, "w")
        text_file.write(listContent[i])
        text_file.close()

In [ ]:
#Generamos el FASTA desde Pfam
if flagPFcode:
    try:
        urlFasta='https://pfam.xfam.org/family/alignment/download/format?acc='+pfam+'&alnType=full&format=fasta&order=t&case=l&gaps=default&download=1'
        ssl._create_default_https_context = ssl._create_unverified_context
        filename = wget.download(urlFasta)
        print("File Fasta downloaded:")
        print(filename)
    except:
        print("Fail downloading Fasta")
        sys.exit()
else:
    filename=param

In [ ]:
#filename='PF00806_full.txt'
filename

In [ ]:
#df_DP_TR = pd.read_csv('SP_TR.txt', sep=" ",header=None)
df_DP_TR = pd.read_csv('SP_collection.txt', sep=" ",header=None)
df_DP_TR.rename({0: 'uniprotAccession', 1: 'uniprotId'}, axis=1, inplace=True)
df_DP_TR.head()

In [ ]:
file1 = open(filename, 'r')
linesDocInput = file1.readlines()
 
tipoDoc=1 #1:Fasta 2:Stocolmo
seqIds=[]
count = 0
encabezados=[]
ids=[]
secuencias=[]
flagSeq=True
lineSeq=''

for line in linesDocInput:
    count += 1

    if (count==1):
        posTipoDoc=line.find('#')
        if posTipoDoc>=0:
            tipoDoc=2
            print("Input in Stockholm format")
        else:
            print("Input in Fasta format")
    
    if tipoDoc==1:
        posHeaderFlag=line.find('>')
        if (posHeaderFlag>=0): #Es encabezado
            if count==1:
                flagSeq=False
            else:
                flagSeq=True
            fin=line.find('/')
            sub=(line.strip())[1:fin]        

            encabezados.append(line.strip())
            ids.append(sub.strip())
        else:
            if flagSeq:
                secuencias.append(lineSeq.strip())
                lineSeq=''
                flagSeq=False                
            lineSeq=lineSeq+line.strip() #concatena las secuencias encontradas
            

file1.close()

In [ ]:
dfGeneral = pd.DataFrame(list(zip(encabezados, ids, secuencias)),
               columns =['encabezado', 'id', 'secuencia'])
dfGeneral.head()
dfGeneral.shape

In [ ]:
#Convertir a DF
dfAccessions = pd.DataFrame(ids, columns =['ids2search'])
dfAccessions=dfAccessions.drop_duplicates()
print("N° items:",dfAccessions.shape)
dfAccessions.head()

In [ ]:
#Obtener los uniprot accessions a partir de los IDs
merged_inner = pd.merge(left=df_DP_TR, right=dfAccessions, left_on='uniprotId', right_on='ids2search')
merged_inner

In [ ]:
#lo convertimos a lista
accessions2search = merged_inner['uniprotAccession'].tolist()
accessions2search

In [ ]:
def obtenerInfoEstructura(rsp):
    #Obtenemos la entrada de Uniprot para poder linkearlo a su PDB
    uniprot_acc=""
    interpro_name=""
    pfam=""
    pdb_accession=""
    uniprot_id=""

    uniprot_id=rsp["responseHeader"]["params"]["q"]
    numPDBFound=rsp["response"]["numFound"]
    print(uniprot_id)

    if(numPDBFound!=0):
        uniprot_acc=rsp["response"]["docs"][0]["entry_uniprot_accession"][0]
        #interpro_name=rsp["response"]["docs"][0]["interpro_repeat_name"][0]
        pfam=rsp["response"]["docs"][0]["pfam"][0]
        pdb_accession=rsp["response"]["docs"][0]["pdb_accession"]

        print("Uniprot accession: ", uniprot_acc)
        print("Interpro Name: ", interpro_name)
        print("Num. PDB found: ", numPDBFound)
        print("PDB accession: ", pdb_accession)
        print("PFAM: ", pfam)
        print("---------")

        PDBCodeStructure.append(pdb_accession)
        
        return True
    else:
        print("--")
        print("No se obtuvieron más datos")
        print("---------")
    
    return False

In [ ]:
#dat='BAT1_PARRH'
#dat='BAT1_MYCRK'
#seq=['BAT1_MYCRK','BAT1_PARRH']

conEstructura=[]
for dat in accessions2search:
    url='https://www.ebi.ac.uk/pdbe/search/pdb/select?q=uniprot_accession:'+dat+'&wt=json'
    response = requests.get(url, data=[])
    rsp=response.json()
    #rsp
    if (obtenerInfoEstructura(rsp)):
        conEstructura.append(dat)
        
print("Tienen estructura: ", conEstructura)

In [ ]:
#Convertir a DF
df_conEstructura = pd.DataFrame(conEstructura, columns =['accessionsEstructura'])
df_conEstructura.shape

In [ ]:
#Obtener los uniprot ID a partir de los accessions
merged_inner2 = pd.merge(left=df_DP_TR, right=df_conEstructura, left_on='uniprotAccession', right_on='accessionsEstructura')
merged_inner2.shape

In [ ]:
dfGeneral.head()

In [ ]:
#Obtenemos los datos de encabezados y secuencia de los que tienen estructura
#dfConEstructuras = dfGeneral[dfGeneral.id.isin(merged_inner2['uniprotId'])]
#dfConEstructuras.shape

In [ ]:
#Obtenemos los datos de encabezados y secuencia de los que tienen estructura
dfSinEstructuras = dfGeneral[~dfGeneral.id.isin(merged_inner2['uniprotId'])]
dfSinEstructuras.shape

In [ ]:
#Organizamos la información tal cual la estructura inicial
#dfAux=pd.DataFrame()
#dfAux["agrupado"]=dfConEstructuras["encabezado"]+ os.linesep+dfConEstructuras["secuencia"]
#dfAux.head()
#dfConEstructuras2=dfConEstructuras.copy()
#dfConEstructuras2['secuencia'] = dfConEstructuras2['secuencia'].apply(correctSeq)
#dfConEstructuras2.shape

In [ ]:
#generar archivos
#df2fastaIndividual(dfConEstructuras2, 2)

In [ ]:
#Generamos el archivo
#print(pfam)
#nombre=pfam
#if (dfAux.size!=0):
    #np.savetxt('target/'+nombre+'_target.fasta', dfAux.values, fmt='%s')
#else:
    #print("No se obtuvo ninguna secuencia que tenga estructura en PDB")

In [ ]:
#=====================================================================================
#=====================================================================================

In [ ]:
#PROCESAMOS LAS SECUENCIAS QUE NO TIENEN ESTRUCTURAS

#Alternativa 1 (IN): Eliminacion o union de repetidas con mejor inicio y fin-> se saca el uniprot_accession, se buscan en archivos el uniprot_id y se busca la secuencia completa
#Alternativa 2 (IS): Un archivo fasta por cada secuencia (check)

In [ ]:
#Alternativa1:
dfSinEstructuras1=dfSinEstructuras.copy()
dfSinEstructuras1.shape
dfSinEstructuras1.head()

In [ ]:
def getIniIndexSeq(x):
    pos1=x.find('/')
    pos2=x.find('-')
    return x[pos1+1:pos2]

In [ ]:
def getEndIndexSeq(x):
    pos1=x.find('-')
    pos2=len(x)
    return x[pos1+1:pos2]

In [ ]:
dfSinEstructuras1['iniIndex'] = dfSinEstructuras1['encabezado']. apply(getIniIndexSeq)
dfSinEstructuras1['endIndex'] = dfSinEstructuras1['encabezado']. apply(getEndIndexSeq)
dfSinEstructuras1.head()
dfSinEstructuras1.shape

In [ ]:
merged_innerAux=pd.DataFrame(dfSinEstructuras1, columns =['id','iniIndex','endIndex'])
merged_innerAux['iniIndex'] = pd.to_numeric(merged_innerAux['iniIndex'])
merged_innerAux['endIndex'] = pd.to_numeric(merged_innerAux['endIndex'])
merged_innerAux.dtypes
merged_innerAux.shape
merged_innerAux.head()

In [ ]:
auxMin=merged_innerAux.groupby(['id']).min()
auxMin=pd.DataFrame(auxMin, columns =['iniIndex'])
auxMin.reset_index(level=0, inplace=True)
auxMin.shape
auxMin.head()

In [ ]:
auxMax=merged_innerAux.groupby(['id']).max()
auxMax=pd.DataFrame(auxMax, columns =['endIndex'])
auxMax.reset_index(level=0, inplace=True)
auxMax.shape

In [ ]:
merged_inner4 = pd.merge(left=auxMin, right=auxMax, left_on='id', right_on='id')
merged_inner4.head()
#merged_inner4.shape

In [ ]:
#prueba=merged_inner4.copy()
#prueba["resta"]=prueba["endIndex"]-prueba["iniIndex"]
#prueba.head()
#prueba["resta2"]=prueba["resta"]>500
#prueba[prueba["resta"]>500]

In [ ]:
#lo convertimos a lista
id_seqs = merged_inner4['id'].tolist()
iniIndexSeq = merged_inner4['iniIndex'].tolist()
endIndexSeq = merged_inner4['endIndex'].tolist()
id_seqs[0]

In [ ]:
counter=0
for item in id_seqs:
    url='https://www.uniprot.org/uniprot/'+item+'.fasta'
    rspta = requests.get(url, data=[])
    #print(rspta.text)
    if (len(rspta.text)!=0):
        rptaFile=response2fasta(basePath+'target/nr_'+item+'_'+str(counter)+'.fasta', rspta, iniIndexSeq[counter], endIndexSeq[counter])  #nr: no repeat
        if rptaFile:
            counter += 1

In [ ]:
#==================================================================================================

In [ ]:
#Alternativa2:
dfSinEstructuras2=dfSinEstructuras.copy()
dfSinEstructuras2.shape

In [ ]:
dfSinEstructuras2['secuencia'] = dfSinEstructuras2['secuencia']. apply(correctSeq)
dfSinEstructuras2

In [ ]:
#generar archivos
df2fastaIndividual(dfSinEstructuras2, 1) 

In [ ]:
#Descargando PDB de la secuencia con los fragmentos con estructura
if len(PDBCodeStructure)>0:
    dirAux=os.path.join(basePath, "auxFiles")
    isdir = os.path.isdir(dirAux) 
        
    if not isdir:        
        try:
            os.makedirs(dirAux)
        except:
            print("Error with directory: "+dirAux)
            sys.exit()

    try:
        urlFasta='https://files.rcsb.org/download/'+PDBCodeStructure[0]+'.pdb'
        filename = wget.download(urlFasta, out=dirAux)
    except:
        print("Fail downloading PDB")